In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp bbox_annotator

In [ ]:
# hide
from nbdev import *

In [ ]:
#exporti

import warnings
import functools
from pathlib import Path
from copy import deepcopy
from typing import Optional, Dict, List

from IPython.core.display import display
from ipywidgets import (AppLayout, Button, HBox, VBox, Layout)

from ipyannotator.base import BaseState, AppWidgetState
from ipyannotator.bbox_canvas import BBoxCanvas, BBoxCanvasState
from ipyannotator.navi_widget import Navi
from ipyannotator.storage import JsonCaptureStorage

# Bounding Box Annotator

Bounding Box Annotator allows users to freely draw on top of images.

## State

In [ ]:
#export

class BBoxState(BaseState):
    coords: Optional[List[Dict[str, float]]]
    image: Optional[str]

## View

In [ ]:
#exporti

class BBoxList(VBox):
    def __init__(
        self, 
        bbox_state: BBoxCanvasState, 
        label_clicked: callable = None
    ):
        super().__init__( 
            layout=Layout(
                min_width='400px'
            )
        )
        
        self._state = bbox_state
        self._label_clicked = label_clicked
        
        if self._state.bbox_coords:
            self._render_btn_list(self._state.bbox_coords)

        bbox_state.subscribe(self._render_btn_list, 'bbox_coords')

    def _render_btn_list(self, bbox_coords: list):
        elements = []
            
        for index, coord in enumerate(bbox_coords):
            btn_annotation = Button(description=f'Annotation {index+1}')
            btn_del_annotation = Button(
                layout=Layout(width='auto'),
                icon="fa-trash", 
                button_style="danger",
                value=index
            )
            
            if self._label_clicked:
                btn_annotation.on_click(
                     functools.partial(self._label_clicked, index=index)
                )
            
            btn_del_annotation.on_click(
                functools.partial(self._del_element, index=index)
            )
            
            elements.append(
                HBox([
                    btn_annotation,
                    btn_del_annotation
                ])
            )
        
        self.children = elements

    def _del_element(self, event: dict, index: int = None):
        if index is not None:
            elements = list(self.children)
            coords = self._state.bbox_coords.copy()
            del coords[index]
            del elements[index]
            self.children = elements
            self._state.bbox_coords = coords

In [ ]:
bbox_state = BBoxCanvasState()

bbox_list = BBoxList(bbox_state)

bbox_list

In [ ]:
# on bbox_state annotation change it reflects on the element list

assert len(bbox_list.children) == 0

bbox_state.bbox_coords = [{'x': 10, 'y': 10, 'width': 20, 'height': 30}]

assert len(bbox_list.children) == 1

# on element click it removes from state

bbox_list.children[0].children[1].click()

assert len(bbox_list.children) == 0

In [ ]:
#exporti

class BBoxAnnotatorGUI(AppLayout):
    def __init__(
        self, 
        app_state: AppWidgetState, 
        bbox_state: BBoxState, 
        save_callable: callable = None
    ):
        self._app_state = app_state
        self._bbox_state = bbox_state
        self._save_callable = save_callable
        
        self._navi = Navi()
        
        self._save_btn = Button(description="Save",
                                layout=Layout(width='auto'))
        
        self._undo_btn = Button(description="Undo",
                                icon="fa-undo",
                                layout=Layout(width='auto'))
        
        self._redo_btn = Button(description="Redo",
                                icon="fa-rotate-right",
                                layout=Layout(width='auto'))

        self._controls_box = HBox(
            [self._navi, self._save_btn, self._undo_btn, self._redo_btn],
            layout=Layout(
                display='flex', 
                flex_flow='row wrap', 
                align_items='center'
            )
        )

        self._image_box = BBoxCanvas(*self._app_state.size)

        self._element_list = BBoxList(
            self._image_box.state, 
            label_clicked=self._highlight_bbox
        )
        
        # set the values already instantiated on state
        if self._app_state.max_im_number:
            self._set_max_im_number(self._app_state.max_im_number)
        
        if self._bbox_state.image:
            self._set_image_path(self._bbox_state.image)
        
        if self._bbox_state.coords:
            self._set_coords(self._bbox_state.coords)
        
        # set the GUI interactions as callables
        self._navi.navi_callable = self._idx_changed
        self._save_btn.on_click(self._save_btn_clicked)
        self._undo_btn.on_click(self._undo_clicked)
        self._redo_btn.on_click(self._redo_clicked)
        
        bbox_state.subscribe(self._set_image_path, 'image')
        bbox_state.subscribe(self._set_coords, 'coords')
        app_state.subscribe(self._set_max_im_number, 'max_im_number')
        
        super().__init__(header=None,
                 left_sidebar=None,
                 center=self._image_box,
                 right_sidebar=self._element_list,
                 footer=self._controls_box,
                 pane_widths=(2, 8, 0),
                 pane_heights=(1, 4, 1))
        
    def _highlight_bbox(self, event: dict, index: int):
        self._image_box.highlight = index

    def _redo_clicked(self, event: dict):
        self._image_box.redo_bbox()
        
    def _undo_clicked(self, event: dict):
        self._image_box.undo_bbox()

    def _set_image_path(self, image: str):
        self._image_box._state.image_path = image

    def _set_coords(self, coords: list):
        if coords:
            tmp_coords = deepcopy(self._image_box._state.bbox_coords)
            tmp_coords.append(coords)
            self._image_box._state.bbox_coords = coords

    def _set_max_im_number(self, max_im_number: int):
        # sync the navi GUI with AppWidgetState
        self._navi.max_im_num = max_im_number

    def _idx_changed(self, index: int):
        # store the last bbox drawn before index update
        self._bbox_state.coords = self._image_box._state.bbox_coords
        self._app_state.index = index

    def _save_btn_clicked(self, *args):
        if self._save_callable:
            self._save_callable(self._image_box._state.bbox_coords)
        else:
            warnings.warn("Save button click didn't triggered any event.")

    def on_client_ready(self, callback):
        self._image_box.observe_client_ready(callback)

In [ ]:
#hide

app_state = AppWidgetState()
bbox_state = BBoxState()

BBoxAnnotatorGUI(
    app_state=app_state, 
    bbox_state=bbox_state,
)

## Controller

In [ ]:
#exporti

class BBoxAnnotatorController:    
    def __init__(self, app_state: AppWidgetState, bbox_state: BBoxState, storage: JsonCaptureStorage):
        self._app_state = app_state
        self._bbox_state = bbox_state
        self._storage = storage
        self._last_index = 0

        app_state.subscribe(self._idx_changed, 'index')

        self._update_im(self._last_index)
        self._app_state.max_im_number = len(self._storage)
        self._update_coords(self._last_index)
    
    def save_current_annotations(self, coords: dict):
        self._bbox_state.set_quietly('coords', coords)
        self._save_annotations(self._app_state.index)
    
    def _update_im(self, index: int):
        self._bbox_state.image = self._storage.images[index]
            
    def _update_coords(self, index: int): # from annotations
        image_path = str(self._storage.images[index])
        self._bbox_state.coords = self._storage.get(image_path) or {}
        
    def _save_annotations(self, index: int, *args, **kwargs): # to disk
        image_path = str(self._storage.images[index])
        self._storage[image_path] = self._bbox_state.coords
        self._storage.save()

    def _idx_changed(self, index: int):
        """ 
        On index change save an old state and update 
        current image path and bbox coordinates for 
        visualisation
        """
        self._save_annotations(self._last_index)
        self._update_im(index)
        self._update_coords(index)
        self._last_index = index

We have annotation saved in dictionary lile: `{'path/to/imagename.jpg': {'x':0, 'y': 0, 'width': 100, 'heigth': 100}}`

Navi widget has `index` and prev/next buttons to iterate over `max_im_number` of images (todo: change name as we can iterate of any object).

BBoxAnnotator has coupled `index` (with Navi one), and onchange event to update the current image path and label.

On image_path change event BBoxCanvas rerenders new image and label

In [ ]:
# hide
# new index ->  save *old* annotations -> update image -> update coordinates from annotation
#                     | 
#                     |-> _update_annotations -> get current bbox values -> save to self.annotations

In [ ]:
# hide

# storage = JsonAnnotationStorage('../data/projects/bbox')

# controller = BBoxAnnotatorController(
#     app_state=app_state,
#     bbox_state=bbox_state,
#     storage=storage
# )

# assert controller._storage.get_num_images() == 4

In [ ]:
#exporti

class BBoxAnnotator:
    """
    Represents bounding box annotator. 
    
    Gives an ability to itarate through image dataset, 
    draw 2D bounding box annotations for object detection and localization,
    export final annotations in json format
    
    """
    
    def __init__(
        self, 
        project_path: Path,
        input_item,
        output_item,
        annotation_file_path: Path,
        *args, **kwargs
    ):
        self.app_state = AppWidgetState(
            uuid=id(self),
            **{
                'size': (input_item.width, input_item.height),
            }
        )
        
        self.bbox_state = BBoxState(
            uuid=id(self)
        )
        
        self.storage = JsonCaptureStorage(
            im_dir=project_path / input_item.dir,
            annotation_file_path=annotation_file_path
        )
        
        self.controller = BBoxAnnotatorController(
            app_state=self.app_state,
            bbox_state=self.bbox_state,
            storage=self.storage
        )
        
        self.view = BBoxAnnotatorGUI(
            app_state=self.app_state,
            bbox_state=self.bbox_state,
            save_callable=self.controller.save_current_annotations
        )

    def __repr__(self):
        display(self.view)
        return ""
    
    def to_dict(self, only_annotated=True):
        return self.storage.to_dict(only_annotated)       

In [ ]:
# hide
from ipyannotator.base import InputImage, OutputImageBbox

in_p = InputImage(image_dir='pics', image_width=640, image_height=400)
out_p = OutputImageBbox()

In [ ]:
! rm -rf ../data/projects/bbox/results

In [ ]:
# hide
from ipyannotator.storage import construct_annotation_path

project_path = Path('../data/projects/bbox')

anno_file_path = construct_annotation_path(project_path)

In [ ]:
# hide
bb = BBoxAnnotator(
    project_path=Path(project_path), 
    input_item=in_p,
    output_item=out_p,
    annotation_file_path=anno_file_path
)

In [ ]:
#hide

bb

In [ ]:
bb.view._image_box._state.bbox_coords = [{'x': 10, 'y': 10, 'width': 20, 'height': 30}]

In [ ]:
# on element click it hightlight

bb.view._element_list.children[0].children[0].click()

assert bb.view._image_box._state.bbox_selected == 0

# double click unhighlight

bb.view._element_list.children[0].children[0].click()
assert bb.view._image_box._state.bbox_selected is None

In [ ]:
bb.storage.to_dict(False)

In [ ]:
# hide
bb.to_dict()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()